In [1]:
# Make sure libraries are reloaded when this cell is run
%load_ext autoreload
%autoreload 2

from astropy import units as u

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter3D
import copy

In [2]:
# Define the two coplanar debris
a = 7363.274488731282 << u.km
ecc = 0 << u.one
inc = 3.0461376917337066 << u.deg
raan = 0.9767211400638387 << u.deg
argp = 0 << u.deg
nu = 270.0 - 180 << u.deg

deb_1 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 6877.943422637125 << u.km
ecc = 0 << u.one
inc = 5.924145688620425 << u.deg
raan = 0.46450412719997725 << u.deg
argp = 0 << u.deg
nu = 89.66915465609816 -180 << u.deg

deb_2 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

# Init the otv at debris 1
otv = copy.copy(deb_1)

frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
# INCLINATION CHANGE

import CustomManeuvres
inc_change = CustomManeuvres.simple_inc_change(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = inc_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(inc_change)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# PRINTOUT
print('DV USED: ', inc_change.get_total_cost())
print('DT USED: ', inc_change.get_total_time())

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

thrust_location mean:  179.999 deg
thrust_norm 369.5362932333629 m / s
y_thrust 9.280060509482988 m / s
z_thrust -369.4197510875594 m / s
nu:  3.1415752002972726 rad
-1.7453292519800304e-05
0.9999999998476913
otv inc 5.924145688392634 deg
deb_2 inc 5.924145688620425 deg
DV USED:  0.369536293233363 km / s
DT USED:  1572.0003894979204 s


In [4]:
import CustomManeuvres
raan_change = CustomManeuvres.simple_raan_change(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = raan_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(raan_change)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# PRINTOUT
print('DV USED: ', raan_change.get_total_cost())
print('DT USED: ', raan_change.get_total_time())
print('COMBINED DV USED: ', raan_change.get_total_cost() + inc_change.get_total_cost() )

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)


theta 0.05281641413158204 deg
u_final 89.74550118302776 deg
thrust_norm -6.782341561295113 m / s
y_thrust 0.0031260500428149426 m / s
z_thrust 6.782340840880984 m / s
nu:  1.5663544844954291 rad
-0.999990135034713
-0.004441827693251181
otv inc 5.976961592079087 deg
deb_2 inc 5.924145688620425 deg
DV USED:  0.0067823415612951144 km / s
DT USED:  6283.661016782642 s
COMBINED DV USED:  0.3763186347946581 km / s


In [5]:
# Hohmann Transfer

# Calculate a basic homnann
import CustomManeuvres
hoh_phas = CustomManeuvres.hohmann_with_phasing(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = hoh_phas.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Propagate the otv using the calculated thrust sequence
transfer_orbit, otv = otv.apply_maneuver(hoh_phas, intermediate=True)

# PRINTOUT
print('DV USED: ', hoh_phas.get_total_cost())
print('DT USED: ', hoh_phas.get_total_time())

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)
frame.plot(transfer_orbit)

/Users/hugoderohanwillner/Documents/MSc AI/Lab Project/Poliastro_Testing/CustomLowLevel.py:49: NumbaPerformanceWarning:

'@' is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'A', False, aligned=True))



Target Delta:  350.39040493589846 deg
current_delta :  228.10004605094105 deg
dist:  122.29035888495741 deg
t_1:  19834.816463699648 s
mean_anomaly_i:  235.3174056371649 deg
new delta:  -369.60959289009367 deg
DV USED:  0.25508829751312895 km / s
DT USED:  22824.716725907165 s


In [6]:
# Total resources used
total_dv = hoh_phas.get_total_cost() + raan_change.get_total_cost() + inc_change.get_total_cost()
total_time = hoh_phas.get_total_time() + raan_change.get_total_time() + inc_change.get_total_time()

# Printout
print('Total DV: ', total_dv)
print('Total Time: ', total_time)

Total DV:  0.6314069323077871 km / s
Total Time:  30680.37813218773 s
